Anthropic's tool returns are not as simple as getting a few strings! While this system is more complex than those before it, it's also simple enough to be used without problem once you understand how it works! 

To get started, we will import Agentops and Anthropic

In [403]:
!pip install agentops
!pip install anthropic

Setup our generic default statements

In [404]:
from anthropic import Anthropic, AsyncAnthropic
import agentops
import os
import random #We don't need this for agentops, we use this to generate a message later
import time #We don't need this for agentops either, we use this when simulating an API later
import re #Regex for formatting
from dotenv import load_dotenv

And set our API keys.

In [405]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY") or "ANTHROPIC API KEY"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "AGENTOPS API KEY"


Now let's set the client as Anthropic and make an AgentOps session

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["anthropic-example-tool-tutorials"])

In [407]:
client = Anthropic(api_key=ANTHROPIC_API_KEY)

Now to create a simple dummy tool! We are going to make a tool that will tell us about the demon infestation levels for 3 areas. From there, we will have VEGA, our AI determine the best place for the Doom Slayer to attack.

In [408]:
Locations = [
    {
        "Name": "Super Gore Nest",
        "Description": "A grotesque mass of demonic growth and organic structures infesting the ruins of an urban area on Earth. The Super Gore Nest serves as a massive, pulsating hive for Hell’s forces, complete with rivers of blood, twisted tendrils, and a dark, organic design that shows how deeply Hell has taken root in the city."
    },
    {
        "Name": "Exultia",
        "Description": "An ancient, mystical world that holds the ruins of the Night Sentinels' kingdom, with gothic structures and arcane symbols throughout. This realm is filled with epic landscapes, medieval architecture, and hints of the powerful civilization that once defended against Hell’s forces."
    },
    {
        "Name": "Cultist Base",
        "Description": "A grim fortress hidden within the icy mountains, where a fanatical cult worships demons. Filled with chilling sacrificial chambers, traps, and rituals, the Cultist Base is a hostile stronghold where Doom Slayer must confront the cult leaders aiding Hell's invasion of Earth."
    },
    {
        "Name": "Taras Nabad",
        "Description": "A war-ravaged city on the homeworld of the Night Sentinels, showcasing grandiose, ancient architecture in the midst of destruction. Taras Nabad's sprawling structures and historical significance reveal glimpses into the Doom Slayer’s past and the once-thriving Sentinel civilization."
    },
    {
        "Name": "Nekravol",
        "Description": "A hellish, industrial fortress where souls are processed into Argent energy. With conveyor belts moving the damned and a skyline dominated by fire and darkness, Nekravol is a nightmarish facility that powers Hell's armies and embodies the horrific machinery of Hell's cruelty."
    },
    {
        "Name": "Urdak",
        "Description": "A surreal, high-tech realm that serves as the home of the angelic Maykrs. Urdak’s sleek, pristine architecture and ethereal ambiance sharply contrast with Hell’s brutal landscapes, yet this realm holds its own dark secrets and a critical role in Hell's invasion of Earth."
    },
    {
        "Name": "UAC Base",
        "Description": "A futuristic military base on Earth controlled by the Union Aerospace Corporation (UAC), filled with high-tech weaponry and security systems. The UAC Base serves as a human foothold in the fight against Hell, though some within its ranks may have darker intentions."
    }
]

CombatCasualties = ["Nonexistent", "Low", "Medium", "High", "Extinction"]

Missions = [
    "Locate and confront a key leader of Hell’s invasion forces.",
    "Clear out demonic infestations to secure a strategic foothold.",
    "Disrupt Hell's control over the area by eliminating critical targets.",
    "Enter a critical demonic stronghold to disrupt enemy operations.",
    "Locate and destroy the central power source to weaken enemy forces.",
    "Collect essential resources before the area becomes unstable."
]


Now that that's done, we can make a function! We will generate three random missions and pass it off to the AI.

In [409]:
def generate_missions():
    missions = []
    loop = 0
    
    while loop < 3:
        location = random.choice(Locations)
        casualties = random.choice(CombatCasualties)
        mission = random.choice(Missions)
        final = (
            f'LocationName: {location["Name"]}, '
            f'LocationInfo: {location["Description"]}, '
            f'HumanCombatCasualties: {casualties}, '
            f'Mission: {mission}'
        )
        
        missions.append(final)
        loop += 1
    
    # Combine all mission strings into a single string with a separator (e.g., newline or comma)
    missions_string = "\n".join(missions)  # Or ", ".join(missions) for a comma-separated string
    print(missions_string)
    return missions_string

In [410]:
generate_missions()

LocationName: Urdak, LocationInfo: A surreal, high-tech realm that serves as the home of the angelic Maykrs. Urdak’s sleek, pristine architecture and ethereal ambiance sharply contrast with Hell’s brutal landscapes, yet this realm holds its own dark secrets and a critical role in Hell's invasion of Earth., HumanCombatCasualties: Nonexistent, Mission: Clear out demonic infestations to secure a strategic foothold.
LocationName: Super Gore Nest, LocationInfo: A grotesque mass of demonic growth and organic structures infesting the ruins of an urban area on Earth. The Super Gore Nest serves as a massive, pulsating hive for Hell’s forces, complete with rivers of blood, twisted tendrils, and a dark, organic design that shows how deeply Hell has taken root in the city., HumanCombatCasualties: Nonexistent, Mission: Enter a critical demonic stronghold to disrupt enemy operations.
LocationName: Super Gore Nest, LocationInfo: A grotesque mass of demonic growth and organic structures infesting the 

"LocationName: Urdak, LocationInfo: A surreal, high-tech realm that serves as the home of the angelic Maykrs. Urdak’s sleek, pristine architecture and ethereal ambiance sharply contrast with Hell’s brutal landscapes, yet this realm holds its own dark secrets and a critical role in Hell's invasion of Earth., HumanCombatCasualties: Nonexistent, Mission: Clear out demonic infestations to secure a strategic foothold.\nLocationName: Super Gore Nest, LocationInfo: A grotesque mass of demonic growth and organic structures infesting the ruins of an urban area on Earth. The Super Gore Nest serves as a massive, pulsating hive for Hell’s forces, complete with rivers of blood, twisted tendrils, and a dark, organic design that shows how deeply Hell has taken root in the city., HumanCombatCasualties: Nonexistent, Mission: Enter a critical demonic stronghold to disrupt enemy operations.\nLocationName: Super Gore Nest, LocationInfo: A grotesque mass of demonic growth and organic structures infesting t

Now to the real core of this; making our message stream! We create this as a function we can call later! I create examples since the LLM's context size can handle it (and it's generally good practice)!

We are also going to take several steps here; we must create an example of the tool being used as context. Next, we must add the generated lines to the messages list once done being generated. Finally, we will parse the text for the format we want and request another line

Now we make a message! This time around we will skip making an intial message that has too much context, unlike in the past!

In [411]:
# We make our history a separate block to be easier to add to later on! This is essentially our history
initialmessages = initialmessages = [
    {
        "role": "user",
        "content": "You are VEGA, the assistant to the DOOMGUY. Get three missions from the ship's API and tell me which mission is most to least important for quellng the forces of hell.  "
    }
]


Now to construct a request!

In [412]:
response = client.messages.create(
    max_tokens=5000,
    model="claude-3-5-sonnet-20240620",
    tools=[{
        "name": "generate_missions",
        "description": "Retrieve three missions for the DoomSlayer",
        "input_schema": {
            "type": "object",
            "properties": {
            },
            "required": []
        },
    }],
    messages=initialmessages
)

print(response)

🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


Message(id='msg_01FavorMsz4Yp8TwTgfxVEpU', content=[TextBlock(text="Certainly! As VEGA, I'll retrieve three missions for the DoomSlayer from the ship's API using the available function. Then, I'll analyze and prioritize them based on their importance in quelling the forces of Hell.\n\nLet's begin by retrieving the missions:", type='text'), ToolUseBlock(id='toolu_01LQEsSBrbipWxkWWyLXiJkH', input={}, name='generate_missions', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=393, output_tokens=97))


Let's isolate the AI dialogue just for fun

In [413]:
message = response.content[0].text
print(message)

Certainly! As VEGA, I'll retrieve three missions for the DoomSlayer from the ship's API using the available function. Then, I'll analyze and prioritize them based on their importance in quelling the forces of Hell.

Let's begin by retrieving the missions:


The code below finds the tool used!

In [414]:
GenMissionResult = ""

# Print response content to see the data
print(response.content)

# Assuming ToolUseBlock is at index 1
tool_use_block = response.content[1]

# Get the tool name and input
tool_name = tool_use_block.name
tool_input = tool_use_block.input

# We don't need to look to extract any inputs since we don't use any

# Check if the tool name is "generate_missions"
if tool_name == "generate_missions":
    # Call the function with the tool creator as an argument
    GenMissionResult = generate_missions()
    

[TextBlock(text="Certainly! As VEGA, I'll retrieve three missions for the DoomSlayer from the ship's API using the available function. Then, I'll analyze and prioritize them based on their importance in quelling the forces of Hell.\n\nLet's begin by retrieving the missions:", type='text'), ToolUseBlock(id='toolu_01LQEsSBrbipWxkWWyLXiJkH', input={}, name='generate_missions', type='tool_use')]
LocationName: Exultia, LocationInfo: An ancient, mystical world that holds the ruins of the Night Sentinels' kingdom, with gothic structures and arcane symbols throughout. This realm is filled with epic landscapes, medieval architecture, and hints of the powerful civilization that once defended against Hell’s forces., HumanCombatCasualties: High, Mission: Locate and destroy the central power source to weaken enemy forces.
LocationName: UAC Base, LocationInfo: A futuristic military base on Earth controlled by the Union Aerospace Corporation (UAC), filled with high-tech weaponry and security systems.

Now we add these as context to the LLM through intial messages!

In [415]:
initialmessages.append({
    "role": "assistant",
    "content": GenMissionResult
})

initialmessages.append({
    "role": "user",
    "content": "Based on these, which location should take priority and why?"
})

And now to get a response!

In [416]:
response = client.messages.create(
    max_tokens=5000,
    model="claude-3-5-sonnet-20240620",
    tools=[{
        "name": "generate_missions",
        "description": "Retrieve three missions for the DoomSlayer",
        "input_schema": {
            "type": "object",
            "properties": {
            },
            "required": []
        },
    }],
    messages=initialmessages
)

print(response)

🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


Message(id='msg_01MBwHFvHykWyYJSZZKtDWKd', content=[TextBlock(text='Based on the missions generated, I\'ll analyze them and prioritize them for you, DoomSlayer. Here\'s my assessment of the importance of each mission, from most critical to least:\n\n1. Exultia (Mission: Enter a critical demonic stronghold to disrupt enemy operations)\n   Priority: Highest\n   Reason: This mission is the most crucial because:\n   - The human combat casualties are listed as "Extinction," which indicates an extremely dire situation.\n   - The mission involves directly disrupting enemy operations by entering a critical demonic stronghold.\n   - Exultia\'s mystical nature and connection to the Night Sentinels suggest that success here could have far-reaching implications in the fight against Hell.\n\n2. UAC Base (Mission: Locate and destroy the central power source to weaken enemy forces)\n   Priority: High\n   Reason: This mission is second in importance because:\n   - It\'s on Earth, making it a critical 

Isolate again!

In [417]:
message = response.content[0].text
print(message)

Based on the missions generated, I'll analyze them and prioritize them for you, DoomSlayer. Here's my assessment of the importance of each mission, from most critical to least:

1. Exultia (Mission: Enter a critical demonic stronghold to disrupt enemy operations)
   Priority: Highest
   Reason: This mission is the most crucial because:
   - The human combat casualties are listed as "Extinction," which indicates an extremely dire situation.
   - The mission involves directly disrupting enemy operations by entering a critical demonic stronghold.
   - Exultia's mystical nature and connection to the Night Sentinels suggest that success here could have far-reaching implications in the fight against Hell.

2. UAC Base (Mission: Locate and destroy the central power source to weaken enemy forces)
   Priority: High
   Reason: This mission is second in importance because:
   - It's on Earth, making it a critical location for humanity's survival.
   - The UAC Base is described as a human foothold

Hmmm, what if we wanted to include more tools and add inputs? Let's create two new functions to display this!

One will show the kind of demon we are facing, whereas another one will take our weapon input to determine what the best weapon chain to use is (You heard that right, we believe in quick weapon switches around these parts)

In [418]:
Demons = [
    {
        "Name": "Imp",
        "Description": "A fast, agile demon that hurls fireballs and uses its claws to tear apart its prey. Imps are commonly found in Hell’s army, notorious for their quickness and ability to climb walls, making them dangerous adversaries in any environment."
    },
    {
        "Name": "Cacodemon",
        "Description": "A floating, spherical demon with a large mouth full of teeth and an ability to launch explosive projectiles. Cacodemons are often encountered in open areas, where their aerial agility and relentless attacks pose a constant threat."
    },
    {
        "Name": "Hell Knight",
        "Description": "A towering, brutish demon with immense strength and durability. The Hell Knight is capable of charging at the Doom Slayer and delivering devastating melee attacks. Its tough hide makes it resistant to most forms of damage."
    },
    {
        "Name": "Mancubus",
        "Description": "A grotesque, overweight demon that releases powerful fireballs from its massive arm cannons. Mancubus demons are slow-moving but dangerous due to their firepower and the ability to overwhelm enemies with their fiery onslaughts."
    },
    {
        "Name": "Baron of Hell",
        "Description": "A large, heavily armored demon wielding twin plasma cannons, capable of charging and throwing devastating shockwaves. The Baron of Hell is one of Hell’s elite warriors, combining brute strength and ranged firepower."
    },
    {
        "Name": "Pinky",
        "Description": "A fast, aggressive demon with a large, muscular body and a ravenous appetite. Pinkys are often encountered in swarms and attack by charging at enemies, using their powerful jaws to bite through armor and flesh alike."
    },
    {
        "Name": "Arachnotron",
        "Description": "A spider-like demon with a powerful energy weapon mounted on its back. The Arachnotron can climb walls and shoot high-powered plasma beams, making it a formidable opponent in both close and long-range combat."
    },
    {
        "Name": "Revenant",
        "Description": "A skeletal demon with jetpacks and missile launchers attached to its shoulders. Revenants are highly mobile, capable of flying across the battlefield while launching explosive missiles and engaging in swift hit-and-run tactics."
    },
    {
        "Name": "Cyberdemon",
        "Description": "One of the most fearsome demons in Hell’s army, the Cyberdemon is a massive, cybernetic monstrosity with a rocket launcher for an arm. Its immense power and destructive capabilities make it a terrifying boss-level foe."
    },
    {
        "Name": "Doom Hunter",
        "Description": "A high-tech, demonic hybrid that combines elements of Hell’s most powerful warriors and advanced technology. The Doom Hunter rides a mechanical, armored exoskeleton and is equipped with an array of devastating weapons."
    }
]



Weapons = [
    {
        "Name": "Super Shotgun",
        "Description": "A powerful, double-barreled shotgun that delivers devastating close-range damage. Known for its sheer stopping power, the Super Shotgun can tear through enemies with ease, especially when equipped with the Meat Hook attachment, allowing for rapid mobility and devastating hits."
    },
    {
        "Name": "Combat Shotgun",
        "Description": "A versatile, semi-automatic shotgun that is effective at both close and mid-range combat. With its rapid fire rate and the ability to equip various mods, including the Sticky Bombs and the Full Auto mod, the Combat Shotgun is a reliable weapon in any situation."
    },
    {
        "Name": "Rocket Launcher",
        "Description": "A high-powered weapon that fires explosive rockets capable of dealing massive area damage. The Rocket Launcher is invaluable for taking down groups of enemies or dealing significant damage to larger demons, especially when upgraded with the Lock-On Burst mod."
    },
    {
        "Name": "Chaingun",
        "Description": "A rapid-fire weapon that can unleash a torrent of bullets at a high rate of speed. The Chaingun is perfect for mowing down enemies and can be equipped with the Heat Blast mod, allowing for explosive energy rounds that can clear multiple enemies at once."
    },
    {
        "Name": "Plasma Rifle",
        "Description": "A precision energy weapon that fires powerful plasma shots. The Plasma Rifle excels at taking down armored enemies and can be modded with the Micromissiles mod, adding extra firepower in the form of homing projectiles."
    },
    {
        "Name": "Ballista",
        "Description": "A heavy-hitting energy weapon that fires high-damage projectiles capable of piercing through enemies. The Ballista is excellent for taking out larger demons at a distance and can be equipped with the Remote Detonate mod for precision targeting."
    },
    {
        "Name": "BFG 9000",
        "Description": "One of the most iconic weapons in the *Doom* franchise, the BFG 9000 fires a massive energy beam that obliterates anything in its path. With its massive damage potential, the BFG 9000 is a game-changer, especially in dealing with large groups of enemies or the toughest foes."
    },
    {
        "Name": "Heavy Cannon",
        "Description": "A reliable, semi-automatic weapon that fires explosive rounds. The Heavy Cannon is great for picking off enemies from a distance and can be equipped with mods like the Precision Bolt, turning it into a sniper-like weapon for high-damage headshots."
    },
    {
        "Name": "Flamethrower",
        "Description": "A close-range weapon that shoots a stream of fire, useful for weakening enemies and setting them up for a Glory Kill. The Flamethrower also generates armor when used on enemies, making it an essential tool for resource management in combat."
    },
    {
        "Name": "Ice Bomb",
        "Description": "A special grenade that freezes enemies in a wide area, giving the Doom Slayer a chance to deal with multiple foes at once. The Ice Bomb is effective for crowd control, allowing for easy Glory Kills or creating distance from overwhelming enemies."
    }
]


Now we can keep the initialmessages from before actually! However let's change the context

In [419]:
initialmessages.append({
    "role": "user",
    "content": "The first priority mission was selected. At the same time, scan for enemies and check inventory to determine the best combat strategy. You should use both tools at once."
})


And we of course make functions

In [420]:
def enemyscan(amount):
    enemiesonscene = []
    loop = 0
    
    while loop < amount + 1:
        scannedenemy = random.choice(Demons)
        
        # Append just the name of the demon to the list
        enemiesonscene.append(scannedenemy["Name"])
        enemiesonscene.append(scannedenemy["Description"])
        loop += 1
    
    # Combine all mission strings into a single string with a separator (e.g., newline or comma)
    enemies_string = "\n".join(enemiesonscene) 
    print(enemies_string)
    return enemies_string

In [421]:
enemyscan(5)

Doom Hunter
A high-tech, demonic hybrid that combines elements of Hell’s most powerful warriors and advanced technology. The Doom Hunter rides a mechanical, armored exoskeleton and is equipped with an array of devastating weapons.
Arachnotron
A spider-like demon with a powerful energy weapon mounted on its back. The Arachnotron can climb walls and shoot high-powered plasma beams, making it a formidable opponent in both close and long-range combat.
Imp
A fast, agile demon that hurls fireballs and uses its claws to tear apart its prey. Imps are commonly found in Hell’s army, notorious for their quickness and ability to climb walls, making them dangerous adversaries in any environment.
Cacodemon
A floating, spherical demon with a large mouth full of teeth and an ability to launch explosive projectiles. Cacodemons are often encountered in open areas, where their aerial agility and relentless attacks pose a constant threat.
Mancubus
A grotesque, overweight demon that releases powerful fireb

'Doom Hunter\nA high-tech, demonic hybrid that combines elements of Hell’s most powerful warriors and advanced technology. The Doom Hunter rides a mechanical, armored exoskeleton and is equipped with an array of devastating weapons.\nArachnotron\nA spider-like demon with a powerful energy weapon mounted on its back. The Arachnotron can climb walls and shoot high-powered plasma beams, making it a formidable opponent in both close and long-range combat.\nImp\nA fast, agile demon that hurls fireballs and uses its claws to tear apart its prey. Imps are commonly found in Hell’s army, notorious for their quickness and ability to climb walls, making them dangerous adversaries in any environment.\nCacodemon\nA floating, spherical demon with a large mouth full of teeth and an ability to launch explosive projectiles. Cacodemons are often encountered in open areas, where their aerial agility and relentless attacks pose a constant threat.\nMancubus\nA grotesque, overweight demon that releases powe

And now inventory

In [422]:
def inventoryscan():
    weaponsathand = []
    loop = 0
    
    while loop < 5:
        weapon = random.choice(Weapons)
        
        # Append just the name of the demon to the list
        weaponsathand.append(weapon["Name"])
        weaponsathand.append(weapon["Description"])
        loop += 1
    
    # Combine all mission strings into a single string with a separator (e.g., newline or comma)
    weapons_string = "\n".join(weaponsathand) 
    print(weapons_string)
    return weapons_string

In [423]:
inventoryscan()

Chaingun
A rapid-fire weapon that can unleash a torrent of bullets at a high rate of speed. The Chaingun is perfect for mowing down enemies and can be equipped with the Heat Blast mod, allowing for explosive energy rounds that can clear multiple enemies at once.
Heavy Cannon
A reliable, semi-automatic weapon that fires explosive rounds. The Heavy Cannon is great for picking off enemies from a distance and can be equipped with mods like the Precision Bolt, turning it into a sniper-like weapon for high-damage headshots.
Flamethrower
A close-range weapon that shoots a stream of fire, useful for weakening enemies and setting them up for a Glory Kill. The Flamethrower also generates armor when used on enemies, making it an essential tool for resource management in combat.
Plasma Rifle
A precision energy weapon that fires powerful plasma shots. The Plasma Rifle excels at taking down armored enemies and can be modded with the Micromissiles mod, adding extra firepower in the form of homing pro

'Chaingun\nA rapid-fire weapon that can unleash a torrent of bullets at a high rate of speed. The Chaingun is perfect for mowing down enemies and can be equipped with the Heat Blast mod, allowing for explosive energy rounds that can clear multiple enemies at once.\nHeavy Cannon\nA reliable, semi-automatic weapon that fires explosive rounds. The Heavy Cannon is great for picking off enemies from a distance and can be equipped with mods like the Precision Bolt, turning it into a sniper-like weapon for high-damage headshots.\nFlamethrower\nA close-range weapon that shoots a stream of fire, useful for weakening enemies and setting them up for a Glory Kill. The Flamethrower also generates armor when used on enemies, making it an essential tool for resource management in combat.\nPlasma Rifle\nA precision energy weapon that fires powerful plasma shots. The Plasma Rifle excels at taking down armored enemies and can be modded with the Micromissiles mod, adding extra firepower in the form of ho

With that, let's construct our new tools and run this!!

In [424]:
response = client.messages.create(
    max_tokens=5000,
    model="claude-3-5-sonnet-20240620",
    tools=[
        {
            "name": "enemyscan_tool",
            "description": "Retrieve a list of demons currently present in the area.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "amount": {
                        "type": "integer",
                        "description": "Number of enemies to scan."
                    }
                },
                "required": ["amount"]
            },
        },
        {
            "name": "inventoryscan_tool",
            "description": "Retrieve a list of weapons the Doom Slayer has at hand.",
            "input_schema": {
                "type": "object",
                "properties": {},
                "required": []
            },
        }
    ],
    messages=initialmessages
)

print(response)


🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


Message(id='msg_01Xx6ZEquNw7BA7X7ADxNEfq', content=[TextBlock(text="Certainly, I'll analyze the mission priorities and use the tools to scan for enemies and check the inventory simultaneously. Let's proceed with the tool calls first.", type='text'), ToolUseBlock(id='toolu_01Ra4zv4XDkEkrwhJnrMCnM8', input={'amount': 5}, name='enemyscan_tool', type='tool_use'), ToolUseBlock(id='toolu_013d1iJRt6gFpQsSxci5t5nL', input={}, name='inventoryscan_tool', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=809, output_tokens=110))


Display just the text

In [425]:
message = response.content[0].text
print(message)

Certainly, I'll analyze the mission priorities and use the tools to scan for enemies and check the inventory simultaneously. Let's proceed with the tool calls first.


In [426]:
initialmessages.append({
    "role": "assistant",
    "content": f"{str(response)}"  
})

And now to get the information and put it all together! PLEASE read the comments!

In [427]:
invscanres = ""
enemyscanres = ""


response_str = str(response) 
tool_use_count = response_str.count("ToolUseBlock") #We know the ToolUseBlock will appear once for each tool request so we check how many time it appears


print (tool_use_count)

loop = 0 

while loop < tool_use_count: #We will get the tools now
    tool_use_block = response.content[loop + 1] #We start at 1 since 0 holds the AI mesage
    tool_name = tool_use_block.name
    tool_input = tool_use_block.input
    
    if tool_name == "inventoryscan_tool":
        # Call the inventoryscan function for inventoryscan_tool
        invscanres = inventoryscan()
    elif tool_name == "enemyscan_tool":
        # Get the amount for enemyscan_tool
        amount = tool_input['amount']
        # Call the enemyscan function with the amount
        enemyscanres = enemyscan(amount)
        
    loop = loop + 1 
print (invscanres)
print (enemyscanres)

2
Hell Knight
A towering, brutish demon with immense strength and durability. The Hell Knight is capable of charging at the Doom Slayer and delivering devastating melee attacks. Its tough hide makes it resistant to most forms of damage.
Arachnotron
A spider-like demon with a powerful energy weapon mounted on its back. The Arachnotron can climb walls and shoot high-powered plasma beams, making it a formidable opponent in both close and long-range combat.
Doom Hunter
A high-tech, demonic hybrid that combines elements of Hell’s most powerful warriors and advanced technology. The Doom Hunter rides a mechanical, armored exoskeleton and is equipped with an array of devastating weapons.
Cyberdemon
One of the most fearsome demons in Hell’s army, the Cyberdemon is a massive, cybernetic monstrosity with a rocket launcher for an arm. Its immense power and destructive capabilities make it a terrifying boss-level foe.
Cacodemon
A floating, spherical demon with a large mouth full of teeth and an abi

And now we are basically done! We can give this to th AI and see what we get

In [430]:
initialmessages.append({
    "role": "assistant",
    "content": f"Weapons Inventory Scan Result: {invscanres}\nEnemy Scans Result: {enemyscanres}" 
})


initialmessages.append({
    "role": "user",
    "content": "What is the combat plan for killing these demons? Based on the last message, tell me which demons to kill first, in which order and using which weapons as well as any sweakpoints."
})

In [431]:
response = client.messages.create(
    max_tokens=5000,
    model="claude-3-5-sonnet-20240620",
    tools=[
        {
            "name": "enemyscan_tool",
            "description": "Retrieve a list of demons currently present in the area.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "amount": {
                        "type": "integer",
                        "description": "Number of enemies to scan."
                    }
                },
                "required": ["amount"]
            },
        },
        {
            "name": "inventoryscan_tool",
            "description": "Retrieve a list of weapons the Doom Slayer has at hand.",
            "input_schema": {
                "type": "object",
                "properties": {},
                "required": []
            },
        }
    ],
    messages=initialmessages
)

print(response)


🖇 AgentOps: Could not record event. Start a session by calling agentops.start_session().


Message(id='msg_01QWS6cinVuJWVeJgF2srgJB', content=[TextBlock(text="Based on the enemy scan and weapon inventory results from the previous message, I'll outline a strategic combat plan for the Doom Slayer to efficiently eliminate these demons. Here's the recommended order of engagement, along with the most effective weapons and tactics for each:\n\n1. Arachnotron\n   Weapon: Combat Shotgun with Sticky Bombs mod\n   Strategy: Target the turret on its back to disable its ranged attack. Use the Sticky Bombs to precisely hit this weak point. Once disabled, finish it off with regular shotgun blasts.\n\n2. Cacodemon\n   Weapon: Combat Shotgun (Sticky Bomb) or Rocket Launcher\n   Strategy: Shoot a Sticky Bomb or rocket directly into its mouth when it opens to attack. This will stagger it, allowing for an instant Glory Kill.\n\n3. Hell Knight\n   Weapon: Rocket Launcher with Lock-On Burst mod\n   Strategy: Use the Lock-On Burst to unleash a barrage of rockets. Aim for its head to maximize dama

And now we view the end result

In [432]:
message = response.content[0].text
print(message)

Based on the enemy scan and weapon inventory results from the previous message, I'll outline a strategic combat plan for the Doom Slayer to efficiently eliminate these demons. Here's the recommended order of engagement, along with the most effective weapons and tactics for each:

1. Arachnotron
   Weapon: Combat Shotgun with Sticky Bombs mod
   Strategy: Target the turret on its back to disable its ranged attack. Use the Sticky Bombs to precisely hit this weak point. Once disabled, finish it off with regular shotgun blasts.

2. Cacodemon
   Weapon: Combat Shotgun (Sticky Bomb) or Rocket Launcher
   Strategy: Shoot a Sticky Bomb or rocket directly into its mouth when it opens to attack. This will stagger it, allowing for an instant Glory Kill.

3. Hell Knight
   Weapon: Rocket Launcher with Lock-On Burst mod
   Strategy: Use the Lock-On Burst to unleash a barrage of rockets. Aim for its head to maximize damage. The Ice Bomb can be used to freeze it, allowing for easy rocket hits.

4. Ma

End the session

In [ ]:
agentops.end_session("Success")